In [ ]:
# Load model directly
cache_dir = "/kaggle/working/my_model_dir"
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base", cache_dir=cache_dir)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)

In [ ]:
cache_dir = "/kaggle/working/my_data_dir"

ds = load_dataset("Kyudan/MathBridge",  cache_dir=cache_dir)

In [ ]:
ds_train = ds["train"]

In [ ]:
def preprocess_data(examples):
    before = examples["context_before"]
    after = examples["context_after"]
    equation = examples["equation"]
    spoken_English = examples["spoken_English"]

    # Prepend a task-specific prompt if necessary, e.g., "translate English to LaTeX:"
    inputs = [f"{before} {spoken_English} {after}"]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    outputs = [f"{before} {equation} {after}"]
    
    with tokenizer.as_target_tokenizer():
        model_outputs = tokenizer(outputs, max_length=512, truncation=True)

    model_inputs["labels"] = model_outputs["input_ids"]

    return model_inputs

In [ ]:
ds_train_preprocessed = ds_train.map(preprocess_data, remove_columns=ds_train.column_names, batched=True)

In [ ]:
ds_train_preprocessed

In [ ]:
ds_train_preprocessed.save_to_disk("./preprocess_data")

In [ ]:
!zip -r data.zip /kaggle/working/preprocess_data